<h1><center>Build <a href="https://howsmynhs.co.uk/">HowsMyNHS</a> website</center></h1>

This is the main script used to automatically generate the html for <a href="https://howsmynhs.co.uk/">HowsMyNHS</a>.

In [1]:
import build_website.build_website as bw
import build_website.plot_data as pd

waiting_data = './data/NHSwaiting.npy'
bed_data = './data/NHSbeds.npy'
news_file = './data/NHS_news_items.ods'

<h2>Generate Plots</h2>

In [2]:
#pd.plotWaitingData(waiting_data)
#pd.plotBedData(bed_data)

<h2>Make homepage</h2>

In [3]:
bw.MakeHomepage(waiting_data, bed_data)

Building homepage... Done


<h2>Make all the other pages</h2>

In [6]:
bw.build_trust_pages(waiting_data, bed_data, news_file)

Building trust pages... Done.
